In [41]:
!pip install backtrader
!pip install arch

In [42]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/cfrm523/Final_project')
from datetime import datetime

<ipython-input-42-b11105c7cb64>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('crypto_mar_april.csv')
data['date'] = pd.to_datetime(data['date'])
data = data.drop('Unnamed: 0', axis=1)

df = data.loc[data['date'] >= datetime(2024,3,1,0,0,0)]
df = df.loc[df['date'] <= datetime(2024,3,10,0,0,0)]
df = df.drop_duplicates()
df = df.reset_index(drop=True)

btc = df.loc[df['ticker']=='BTC-USD']
btc = btc.reset_index(drop=True)
btc = btc.set_index('date')

eth = df.loc[df['ticker']=='ETH-USD']
eth = eth.reset_index(drop=True)
eth = eth.set_index('date')

ltc = df.loc[df['ticker']=='LTC-USD']
ltc = ltc.reset_index(drop=True)
ltc = ltc.set_index('date')

sol = df.loc[df['ticker']=='SOL-USD']
sol = sol.reset_index(drop=True)
sol = sol.set_index('date')

In [62]:
from arch.unitroot import ADF
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from sklearn.linear_model import LinearRegression

def strat_creation(datas):
  # Create a Stratey
  class spread_strat(bt.Strategy):

      params = (
          ('history', 120),
          ('lookback', 2)
      )

      def log(self, txt, dt=None):
          ''' Logging function for this strategy '''
          if dt is None:
              # Access the datetime index from the current line in the data series
              dt = self.datas[0].datetime.datetime(0)

          # Check if dt is still a float (the internal representation for Backtrader), and convert it if needed
          if isinstance(dt, float):
              # Convert backtrader float date to datetime
              dt = bt.num2date(dt)

          # Format datetime object to string
          dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')
          print('%s, %s' % (dt_str, txt))

      def close_all_positions(self):
        ## Close all positions
        for data in self.datas:
          self.close(data)

      def __init__(self):
          # Keep a reference to the "close" line in the data[0] dataseries
          if datas == 2:
            self.data0 = self.datas[0].close
            self.data1 = self.datas[1].close
          #constant for strat
          self.cointegrated = False
          self.upper_bound = None
          self.lower_bound = None
          self.model_built = False
          self.coefficients = None

      #trading during best time
      def next(self):
          current_datetime = self.datas[0].datetime.datetime(0)
          current_hour = current_datetime.hour
          current_minute = current_datetime.minute
          if datas == 2:
            series1 = pd.Series(self.data0.get(size=self.params.history))
            series2 = pd.Series(self.data1.get(size=self.params.history))
            series1_diff = series1.diff().dropna()  # taking the diff of both series
            series2_diff = series2.diff().dropna()
            tickers = [series1_diff, series2_diff]
          if 6 <= current_hour < 8: #this is inherently rolling
            if len(self.data0) >= self.params.history and len(self.data1) >= self.params.history: #check if there is sufficient data for looking back
              #######################################
              if self.model_built == False: #we only build the model once for the time period
                for i in range(len(tickers)): #iterate through series and check for stationarity
                  adf = ADF(tickers[i])
                  pp = PhillipsPerron(tickers[i])
                  kpss = KPSS(tickers[i])
                  if adf.pvalue < 0.05 and pp.pvalue < 0.05 and kpss.pvalue > 0.10: #check weather pass all the tests
                    self.cointegrated = True
                  else:
                    self.cointegrated = False
                  #self.log(f'Checking series stationarity: ADF P-Value: {adf.pvalue}, PP P-Value: {pp.pvalue}, KPSS P-Value: {kpss.pvalue}')
                #we want to define the model once and trade on that model for the time period
                if self.cointegrated == True: #2nd step, fit a linear model and create a spread
                  lin_model = LinearRegression()
                  lin_model.fit(series1.values.reshape(-1,1), series2.values)  # Fit model to the differenced data
                  spread = 1*series1 - (lin_model.coef_[0] * series2) #value of the spread for model
                  mean = np.mean(spread)#mean
                  self.upper_bound = mean + 1.5*np.std(spread)
                  self.lower_bound = mean - 1.5*np.std(spread)
                  self.coefficients = [1] + list(-1*lin_model.coef_)
                  self.model_built = True
                  self.log(f'Upper bound {self.upper_bound}, Lower bound {self.lower_bound}, coefs:{self.coefficients}')
              ######################################
            #we check if upper or lower bound is crossed to enter a trade
              elif self.model_built == True:
                #need to calculate the value of the spread
                spread_val = self.data0[0] + self.data1[0]*self.coefficients[1]
                #identify if not in position
                if not self.position:
                  #get into a long
                  if spread_val < self.lower_bound:
                    self.log('Buy Created, %.2f' % spread_val)
                    self.buy(self.datas[0], size=1)
                    self.sell(self.datas[1], size=self.coefficients[1])
                  #get into a short
                  elif spread_val > self.upper_bound:
                    self.log('Sell Created, %.2f' % spread_val)
                    self.sell(self.datas[0], size=1)
                    self.buy(self.datas[1], size=self.coefficients[1])
                #identify if in position
                elif self.position:
                  position_size = self.position.size
                  #check if currently in long and hit our exit (upper boundary)
                  if position_size > 0 and spread_val > self.upper_bound:
                      self.log('Close existing Long position, %.2f' % spread_val)
                      self.close_all_positions()
                  #check if currently in short and hit our exit (lower boundary)
                  elif position_size < 0 and spread_val < self.lower_bound:
                      self.log('Close existing Short position, %.2f' % spread_val)
                      self.close_all_positions()

          #reset our model after 6 and close out any positions
          if current_hour == 8 and current_minute == 0: #always close out trades
            self.cointegrated = False
            self.upper_bound = None
            self.lower_bound = None
            self.model_built = False
            self.coefficients = None
            if self.position:
              self.log("Closing all positions: ")
              self.close_all_positions()

          #if current_hour == 8 and current_minute == 1:
          #  for data in self.datas:
          #    pos = self.getposition(data)
          #    self.log(f'size:{pos.size}')

          #for data in self.datas:
          #  pos = self.getposition(data)
          #  if pos.size != 0:
          #    self.log(f'size:{pos.size}')


  return spread_strat

In [63]:
from itertools import combinations

cryptos = ['BTC-USD', 'ETH-USD', 'LTC-USD', 'SOL-USD']
# Collect all combinations from size 2 to the length of the list
all_combinations = []
for r in range(2, len(cryptos)+1):
    all_combinations.extend(list(combinations(cryptos, r)))

spreads = []
# Print all combinations
for combo in all_combinations:
    spreads.append(list(combo))

metrics = {
        'spread': [],
        'Total Net Profit': [],
        'Gross Profit': [],
        'Gross Loss': [],
        'Percent Profitable': [],
        'Winning Trades': [],
        'Losing Trades': [],
        'Avg. Trade Net Profit': [],
        'Avg. Winning Trade': [],
        'Avg. Losing Trade': [],
        'Ratio Avg. Win:Avg. Loss': [],
        'Largest Winning Trade': [],
        'Largest Losing Trade': [],
        'Max. Consecutive Winning Trades': [],
        'Max. Consecutive Losing Trades': [],
        'Avg. Bars in Total Trades': [],
        'Avg. Bars in Winning Trades': [],
        'Avg. Bars in Losing Trades': [],
        'Max. Drawdown': []
    }

spreads = [['BTC-USD', 'ETH-USD']]
for i in spreads:
  print(i)
  cerebro = bt.Cerebro()
  TestStrategy = strat_creation(2)
  cerebro.addstrategy(TestStrategy)
  if 'BTC-USD' in i:
    btc_bt = bt.feeds.PandasData(dataname=btc)
    cerebro.adddata(btc_bt)
  if 'ETH-USD' in i:
    eth_bt = bt.feeds.PandasData(dataname=eth)
    cerebro.adddata(eth_bt)
  if 'LTC-USD' in i:
    ltc_bt = bt.feeds.PandasData(dataname=ltc)
    cerebro.adddata(ltc_bt)
  if 'SOL-USD' in i:
    sol_bt = bt.feeds.PandasData(dataname=sol)
    cerebro.adddata(sol_bt)

  initial_cash = 1000000
  cerebro.broker.set_cash(initial_cash)
  cerebro.broker.setcommission(commission=0.0)

  # Add analyzers
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
  cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio", riskfreerate=0.0)
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
  cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
  # Run backtest
  results = cerebro.run()
  strategy = results[0]

  # Extract metrics from analyzers
  trade_analyzer = strategy.analyzers.trade_analyzer.get_analysis()
  sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis()
  drawdown = strategy.analyzers.drawdown.get_analysis()
  returns = strategy.analyzers.returns.get_analysis()

  # Calculate custom metrics
  total_net_profit = cerebro.broker.getvalue() - initial_cash
  total_gross_profit = trade_analyzer.won.pnl.total
  total_gross_loss = trade_analyzer.lost.pnl.total
  total_number_trades = trade_analyzer.total.total
  percent_profitable = (trade_analyzer.won.total / trade_analyzer.total.total) * 100
  winning_trades = trade_analyzer.won.total
  losing_trades = trade_analyzer.lost.total
  avg_trade_net_profit = total_net_profit / trade_analyzer.total.total
  avg_winning_trade = trade_analyzer.won.pnl.average
  avg_losing_trade = trade_analyzer.lost.pnl.average
  ratio_avg_win_loss = avg_winning_trade/avg_losing_trade
  largest_winning_trade = trade_analyzer.won.pnl.max
  largest_losing_trade = trade_analyzer.lost.pnl.max
  max_consecutive_winning_trades = trade_analyzer.streak.won.longest
  max_consecutive_losing_trades = trade_analyzer.streak.lost.longest
  avg_bars_in_total_trades = trade_analyzer.len.total
  avg_bars_in_winning_trades = trade_analyzer.len.won.total
  avg_bars_in_losing_trades = trade_analyzer.len.lost.total
  max_drawdown = drawdown.max.moneydown

  metrics['spread'].append(i)
  metrics['Total Net Profit'].append(total_net_profit)
  metrics['Gross Profit'].append(total_gross_profit)
  metrics['Gross Loss'].append(total_gross_loss)
  metrics['Percent Profitable'].append(percent_profitable)
  metrics['Winning Trades'].append(winning_trades)
  metrics['Losing Trades'].append(losing_trades)
  metrics['Avg. Trade Net Profit'].append(avg_trade_net_profit)
  metrics['Avg. Winning Trade'].append(avg_winning_trade)
  metrics['Avg. Losing Trade'].append(avg_losing_trade)
  metrics['Ratio Avg. Win:Avg. Loss'].append(ratio_avg_win_loss)
  metrics['Largest Winning Trade'].append(largest_winning_trade)
  metrics['Largest Losing Trade'].append(largest_losing_trade)
  metrics['Max. Consecutive Winning Trades'].append(max_consecutive_winning_trades)
  metrics['Max. Consecutive Losing Trades'].append(max_consecutive_losing_trades)
  metrics['Avg. Bars in Total Trades'].append(avg_bars_in_total_trades)
  metrics['Avg. Bars in Winning Trades'].append(avg_bars_in_winning_trades)
  metrics['Avg. Bars in Losing Trades'].append(avg_bars_in_losing_trades)
  metrics['Max. Drawdown'].append(max_drawdown)

pd.DataFrame.from_dict(metrics).T

['BTC-USD', 'ETH-USD']
2024-03-01 06:00:00, Upper bound 61470.81314678685, Lower bound 61122.71799277306, coefs:[1, -0.0259038467121626]
2024-03-01 06:16:00, Sell Created, 61573.78
2024-03-01 08:00:00, Closing all positions: 
2024-03-02 06:00:00, Upper bound 62034.03944009025, Lower bound 61668.08877235029, coefs:[1, -0.08073012937735784]
2024-03-02 06:21:00, Sell Created, 62044.73
2024-03-02 07:05:00, Close existing Short position, 61657.31
2024-03-02 07:10:00, Buy Created, 61591.39
2024-03-02 08:00:00, Closing all positions: 
2024-03-03 06:21:00, Upper bound 61700.60444082274, Lower bound 61472.654986596324, coefs:[1, -0.11694304475008895]
2024-03-03 07:02:00, Buy Created, 61471.66
2024-03-03 08:00:00, Closing all positions: 
2024-03-04 06:00:00, Upper bound 63540.75353565605, Lower bound 63139.51230091201, coefs:[1, -0.057974669974927145]
2024-03-04 07:20:00, Sell Created, 63542.06
2024-03-04 08:00:00, Closing all positions: 
2024-03-05 06:00:00, Upper bound 68191.91202836219, Lower

,0
spread,"[BTC-USD, ETH-USD]"
Total Net Profit,327.681615
Gross Profit,1443.023819
Gross Loss,-1115.342205
Percent Profitable,50.0
Winning Trades,11
Losing Trades,11
Avg. Trade Net Profit,14.894619
Avg. Winning Trade,131.183984
Avg. Losing Trade,-101.394746
